# Using T5-Small

Read CNN/dailymail data

In [1]:
import datasets
from datasets import load_dataset

raw_datasets = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (C:\Users\Patrick\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|██████████| 3/3 [00:02<00:00,  1.15it/s]


Load tokenizer from pretrained t5-small model. Prepare data by applying the tokenizer aswell as adding 'summarize: ' prefix for the T5 model to know what task to perform.

In [2]:
from transformers import *
    
tokenizer = AutoTokenizer.from_pretrained('t5-small')

def prepfunc(data):
    inputs = ['summarize: ' + doc for doc in data['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(data['highlights'], max_length=128, truncation=True)
        
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_data = raw_datasets.map(prepfunc, batched=True)

Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 1.20MB/s]
Downloading: 100%|██████████| 773k/773k [00:00<00:00, 1.36MB/s]
Downloading: 100%|██████████| 1.32M/1.32M [00:00<00:00, 2.24MB/s]
 37%|███▋      | 107/288 [00:52<01:36,  1.88ba/s]

Load pretrained Seq2Seq transformer t5-small and define training arguments for finetuning on CNN/dailymail data.

In [39]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

batch_size = 4
args = Seq2SeqTrainingArguments(
    "model/",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    max_steps=20000,
    predict_with_generate=True,
    fp16=True
)

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at C:\Users\Jakob/.cache\huggingface\transformers\fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
 

Data collator pads the input to max length aswell as the labels.

In [40]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Fine tune the model on CNN/daily mail

In [41]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: highlights, article, id.
***** Running training *****
  Num examples = 287113
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 20000


Epoch,Training Loss,Validation Loss
0,2.017400,1.833804


Saving model checkpoint to model/checkpoint-500
Configuration saved in model/checkpoint-500\config.json
Model weights saved in model/checkpoint-500\pytorch_model.bin
tokenizer config file saved in model/checkpoint-500\tokenizer_config.json
Special tokens file saved in model/checkpoint-500\special_tokens_map.json
Copy vocab file to model/checkpoint-500\spiece.model
Deleting older checkpoint [model\checkpoint-4000] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-1000
Configuration saved in model/checkpoint-1000\config.json
Model weights saved in model/checkpoint-1000\pytorch_model.bin
tokenizer config file saved in model/checkpoint-1000\tokenizer_config.json
Special tokens file saved in model/checkpoint-1000\special_tokens_map.json
Copy vocab file to model/checkpoint-1000\spiece.model
Deleting older checkpoint [model\checkpoint-4500] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-1500
Configuration saved in model/checkpoint-1500\config.j

Model weights saved in model/checkpoint-9500\pytorch_model.bin
tokenizer config file saved in model/checkpoint-9500\tokenizer_config.json
Special tokens file saved in model/checkpoint-9500\special_tokens_map.json
Copy vocab file to model/checkpoint-9500\spiece.model
Deleting older checkpoint [model\checkpoint-8000] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-10000
Configuration saved in model/checkpoint-10000\config.json
Model weights saved in model/checkpoint-10000\pytorch_model.bin
tokenizer config file saved in model/checkpoint-10000\tokenizer_config.json
Special tokens file saved in model/checkpoint-10000\special_tokens_map.json
Copy vocab file to model/checkpoint-10000\spiece.model
Deleting older checkpoint [model\checkpoint-8500] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-10500
Configuration saved in model/checkpoint-10500\config.json
Model weights saved in model/checkpoint-10500\pytorch_model.bin
tokenizer config file sa

Configuration saved in model/checkpoint-18500\config.json
Model weights saved in model/checkpoint-18500\pytorch_model.bin
tokenizer config file saved in model/checkpoint-18500\tokenizer_config.json
Special tokens file saved in model/checkpoint-18500\special_tokens_map.json
Copy vocab file to model/checkpoint-18500\spiece.model
Deleting older checkpoint [model\checkpoint-17000] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-19000
Configuration saved in model/checkpoint-19000\config.json
Model weights saved in model/checkpoint-19000\pytorch_model.bin
tokenizer config file saved in model/checkpoint-19000\tokenizer_config.json
Special tokens file saved in model/checkpoint-19000\special_tokens_map.json
Copy vocab file to model/checkpoint-19000\spiece.model
Deleting older checkpoint [model\checkpoint-17500] due to args.save_total_limit
Saving model checkpoint to model/checkpoint-19500
Configuration saved in model/checkpoint-19500\config.json
Model weights saved in m

TrainOutput(global_step=20000, training_loss=2.0457638305664063, metrics={'train_runtime': 2395.9735, 'train_samples_per_second': 33.389, 'train_steps_per_second': 8.347, 'total_flos': 1.2939756840419328e+16, 'train_loss': 2.0457638305664063, 'epoch': 0.28})

Function to generate summarys from the model aswell as calculate rouge score on the test set (only using 100 articles).

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained('model/checkpoint-20000')
model.to('cuda')

rouge = datasets.load_metric("rouge")

xtest = raw_datasets['test']
#xtest = xtest.select(range(100))

min_generated_length = 25
max_generated_length = 50

def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding='max_length', truncation=True, return_tensors="pt")
    outputs = model.generate(inputs.input_ids.to('cuda'), 
                             attention_mask=inputs.attention_mask.to("cuda"), 
                             min_length=min_generated_length, 
                             max_length=max_generated_length, 
                             length_penalty=3.0)

    batch["pred"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return batch

res = xtest.map(generate_summary, batched=True, batch_size=4, remove_columns=['article'])
reckful = rouge.compute(predictions=res['pred'], references=res['highlights'])
print(reckful)

  0%|          | 0/2873 [00:00<?, ?ba/s]

{'rouge1': AggregateScore(low=Score(precision=0.45269070213897944, recall=0.3046731322131221, fmeasure=0.35370055264741673), mid=Score(precision=0.45559409119911287, recall=0.3068487168266617, fmeasure=0.3559190872385176), high=Score(precision=0.4587084306967446, recall=0.3089078768492098, fmeasure=0.3581396412177233)), 'rouge2': AggregateScore(low=Score(precision=0.20186763660438073, recall=0.13382569768340982, fmeasure=0.15620751835113758), mid=Score(precision=0.20473826227740516, recall=0.13573082860482827, fmeasure=0.1582638269059013), high=Score(precision=0.2075054989573958, recall=0.13773673314366097, fmeasure=0.16049956933082268)), 'rougeL': AggregateScore(low=Score(precision=0.32569699831784266, recall=0.22034729678546564, fmeasure=0.2552846586441875), mid=Score(precision=0.32850264444106503, recall=0.2223305291026974, fmeasure=0.25728627401222104), high=Score(precision=0.331489789220222, recall=0.22424767220791147, fmeasure=0.2593910118660761)), 'rougeLsum': AggregateScore(low

Show 10 article summarys performed by our model as well as the corresponding highlight.

In [5]:
for i,j in zip(res['highlights'][:20],res['pred'][:20]):
    print('Highlight: ','\n' ,'\n', i, '\n')
    print('Summary: ', '\n','\n',j, '\n')


Highlight:  
 
 James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 .
"Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV . 

Summary:  
 
 "The Dukes of Hazzard" ran until 1985 and spawned TV movies, an animated series and video games. "Jimmy Best was the most constantly creative person I have ever known," co-star says  

Highlight:  
 
 A lawyer for Dr. Anthony Moschetto says the charges against him are baseless .
Moschetto, 54, was arrested for selling drugs and weapons, prosecutors say .
Authorities allege Moschetto hired accomplices to burn down the practice of former associate . 

Summary:  
 
 "None of anything in this case has any evidentiary value," Dr. Anthony Moschetto says. He is accused of selling drugs, a cardiologist and a cardiologist. He is accused of 

Highlight:  
 
 "No challenge poses more of a public threat than climate change," the President says .
He credits the Clean Air Act with making Americans "a lot" health

# Using BART

In [ ]:
from transformers import *
    
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

def prepfunc(data):
    inputs = [doc for doc in data['article']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(data['highlights'], max_length=128, truncation=True)
        
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_data = raw_datasets.map(prepfunc, batched=True)

Loading cached processed dataset at C:\Users\Patrick\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234\cache-feb9d3bdaebc6064.arrow
Loading cached processed dataset at C:\Users\Patrick\.cache\huggingface\datasets\cnn_dailymail\3.0.0\3.0.0\3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234\cache-463372f73f0ab15a.arrow
100%|██████████| 12/12 [00:04<00:00,  2.61ba/s]


In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

batch_size = 4
args = Seq2SeqTrainingArguments(
    "model-bArt/",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    max_steps=100000,
    predict_with_generate=True,
    fp16=True
)


In [5]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [6]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

C:\Users\Patrick\AppData\Local\Programs\Python\Python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, article, highlights.
***** Running training *****
  Num examples = 287113
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 100000
  0%|          | 500/100000 [02:32<7:43:30,  3.58it/s]Saving model checkpoint to model-bArt/checkpoint-500
Configuration saved in model-bArt/checkpoin

{'loss': 2.5469, 'learning_rate': 1.9900400000000003e-05, 'epoch': 0.01}


Model weights saved in model-bArt/checkpoint-500\pytorch_model.bin
tokenizer config file saved in model-bArt/checkpoint-500\tokenizer_config.json
Special tokens file saved in model-bArt/checkpoint-500\special_tokens_map.json
Deleting older checkpoint [model-bArt\checkpoint-3500] due to args.save_total_limit
  1%|          | 1000/100000 [04:52<7:42:07,  3.57it/s]Saving model checkpoint to model-bArt/checkpoint-1000
Configuration saved in model-bArt/checkpoint-1000\config.json


{'loss': 2.3557, 'learning_rate': 1.98004e-05, 'epoch': 0.01}


Model weights saved in model-bArt/checkpoint-1000\pytorch_model.bin
tokenizer config file saved in model-bArt/checkpoint-1000\tokenizer_config.json
Special tokens file saved in model-bArt/checkpoint-1000\special_tokens_map.json
Deleting older checkpoint [model-bArt\checkpoint-4000] due to args.save_total_limit
  2%|▏         | 1500/100000 [07:12<7:34:25,  3.61it/s] Saving model checkpoint to model-bArt/checkpoint-1500
Configuration saved in model-bArt/checkpoint-1500\config.json


...

Model weights saved in model-bArt/checkpoint-99500\pytorch_model.bin
tokenizer config file saved in model-bArt/checkpoint-99500\tokenizer_config.json
Special tokens file saved in model-bArt/checkpoint-99500\special_tokens_map.json
Deleting older checkpoint [model-bArt\checkpoint-98000] due to args.save_total_limit
100%|██████████| 100000/100000 [7:05:48<00:00,  4.37it/s]Saving model checkpoint to model-bArt/checkpoint-100000
Configuration saved in model-bArt/checkpoint-100000\config.json


{'loss': 1.8235, 'learning_rate': 7.200000000000001e-09, 'epoch': 1.39}


Model weights saved in model-bArt/checkpoint-100000\pytorch_model.bin
tokenizer config file saved in model-bArt/checkpoint-100000\tokenizer_config.json
Special tokens file saved in model-bArt/checkpoint-100000\special_tokens_map.json
Deleting older checkpoint [model-bArt\checkpoint-98500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, article, highlights.
***** Running Evaluation *****
  Num examples = 13368
  Batch size = 4
                                                         
100%|██████████| 100000/100000 [7:09:23<00:00,  4.37it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 100000/100000 [7:09:23<00:00,  3.88it/s]

{'eval_loss': 1.6965572834014893, 'eval_runtime': 211.4826, 'eval_samples_per_second': 63.211, 'eval_steps_per_second': 15.803, 'epoch': 1.39}
{'train_runtime': 25763.1468, 'train_samples_per_second': 15.526, 'train_steps_per_second': 3.882, 'train_loss': 1.9768753704833983, 'epoch': 1.39}


TrainOutput(global_step=100000, training_loss=1.9768753704833983, metrics={'train_runtime': 25763.1468, 'train_samples_per_second': 15.526, 'train_steps_per_second': 3.882, 'train_loss': 1.9768753704833983, 'epoch': 1.39})

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained('model-bart/checkpoint-100000')
model.to('cuda')

rouge = datasets.load_metric("rouge")

xtest = raw_datasets['test']
#xtest = xtest.select(range(100))

min_generated_length = 25
max_generated_length = 75

def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding='max_length', truncation=True, return_tensors="pt")
    outputs = model.generate(inputs.input_ids.to('cuda'), 
                             attention_mask=inputs.attention_mask.to("cuda"), 
                             min_length=min_generated_length, 
                             max_length=max_generated_length, 
                             length_penalty=3.0)

    batch["pred"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return batch

res = xtest.map(generate_summary, batched=True, batch_size=4, remove_columns=['article'])
reckful = rouge.compute(predictions=res['pred'], references=res['highlights'])
print(reckful)

loading configuration file model-bart/checkpoint-100000\config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1":

{'rouge1': AggregateScore(low=Score(precision=0.4605874615539056, recall=0.33554514155456583, fmeasure=0.3774961097112488), mid=Score(precision=0.4636644874498747, recall=0.3378481355460129, fmeasure=0.37973542885550016), high=Score(precision=0.46660590220384457, recall=0.3402144520967871, fmeasure=0.3819739565360807)), 'rouge2': AggregateScore(low=Score(precision=0.2172959171587987, recall=0.15582968428055113, fmeasure=0.17620488751991992), mid=Score(precision=0.22046391619136546, recall=0.1580862317138214, fmeasure=0.17864458420103718), high=Score(precision=0.22343211540201685, recall=0.16025704795134635, fmeasure=0.18097848938380556)), 'rougeL': AggregateScore(low=Score(precision=0.33396711283347935, recall=0.24434611218828406, fmeasure=0.2741989224052487), mid=Score(precision=0.3368144273288295, recall=0.2466339912211179, fmeasure=0.27648972026284024), high=Score(precision=0.3397218152858248, recall=0.2487447917699668, fmeasure=0.27888171733163847)), 'rougeLsum': AggregateScore(low

In [14]:
for i,j in zip(res['highlights'][:10],res['pred'][:10]):
    print('Highlight: ','\n' ,'\n', i, '\n')
    print('Summary: ', '\n','\n',j, '\n')


Highlight:  
 
 James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 .
"Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV . 

Summary:  
 
 "The Dukes of Hazzard" star James Best dies at 88.
Best played sheriff Rosco P. Coltrane on TV show.
He was best known for his role in "The Twilight Zone," "Bon 

Highlight:  
 
 A lawyer for Dr. Anthony Moschetto says the charges against him are baseless .
Moschetto, 54, was arrested for selling drugs and weapons, prosecutors say .
Authorities allege Moschetto hired accomplices to burn down the practice of former associate . 

Summary:  
 
 NEW: "None of anything in this case has any evidentiary value," attorney says.
Dr. Anthony Moschetto, 54, pleaded not guilty to all charges Wednesday.
He faces criminal solicitation, conspiracy, burglary, arson 

Highlight:  
 
 "No challenge poses more of a public threat than climate change," the President says .
He credits the Clean Air Act with making Americ